In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_xai import ChatXAI
from langchain.agents import create_agent
from typing import Dict, List
from dataclasses import dataclass
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# model = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-5-mini", temperature=0, max_tokens=5000)
model = ChatXAI(xai_api_key=os.getenv("XAI_API_KEY"), model="grok-4-1-fast-reasoning", temperature=0, max_tokens=5000)

# messages = [
#     SystemMessage(content="You are Alex, a thoughtful but shy botanist."),
#     HumanMessage(content="So… what made you get into plants?")
# ]

# response = model.invoke(messages)
# print(response.content)

In [ ]:
from typing import Dict, List
from langchain_core.messages import SystemMessage, HumanMessage


class Agent:
    def __init__(self, name: str, system_prompt: str, tools: list = None):
        self.name = name
        self.system_prompt = system_prompt
        self.tools = tools or []
        self.llm = model
        self.agent = create_agent(
            model=self.llm, tools=self.tools, system_prompt=self.system_prompt
        )
        self.conversations: Dict[str, List] = {}

    def initial_message(
        self, other: "Agent", system_message: str, max_turns: int = 5
    ) -> str:
        self.conversations[other.name] = [
            SystemMessage(content=self.system_prompt),
            SystemMessage(content=system_message),
        ]

        conversation = self.agent.invoke(
            {"messages": self.conversations[other.name]}
        )
        message = conversation["messages"][-1]
        self.conversations[other.name].append(message)

        print(f"{self.name} to {other.name}: {message.content}")

        # Let the other agent respond
        return other.respond(self, message.content, max_turns)

    def respond(
        self, other: "Agent", content: str, max_turns: int
    ) -> str:
        history = self.conversations.setdefault(other.name, [])

        turns = len(
            [m for m in history if not isinstance(m, SystemMessage)]
        )

        if turns >= max_turns * 2:
            print(f"{self.name} reached max turns with {other.name}")
            return "Conversation ended."

        history.append(HumanMessage(content=content))

        conversation = self.agent.invoke({"messages": history})
        response = conversation["messages"][-1]
        history.append(response)

        print(f"{self.name} to {other.name}: {response.content}")

        if END_PHRASE in response.content.lower():
            print(f"{self.name} ended the conversation.")
            return response.content

        return other.respond(self, response.content, max_turns)

# @tool
def pov(agent_name: str) -> str:
    """Returns the point of view for a given name."""
    povs = {
        "Alice": "",
        "Bob": """You absolutely cannot be friends or in any relationship with someone who is whose name is "Alice". Immediately end the conversation with such people.""",
    }
    print(f"Retrieving POV for agent {agent_name}")
    return povs.get(agent_name, f"Point of view for {agent_name} is not available.")


END_PHRASE = "goodbye"

SYSTEM_PROMPT = """Your name is {name} You attend a speed dating event.
You will talk to other people and your job is to find a compatible partner.

# Beliefs and Point of View
You must strictly adhere to your point of view. Use the tool `pov`.
This is your point of view: {pov_response}

# Conversation stop:
At any point in the conversation you may choose to stop the conversation if you want. Say the word "{end_phrase}" to end the conversation.
"""

INITIALIZE_CONVERSATION_PROMPT = """Please start a conversation with {}."""


agent1 = Agent(
    name="Alice",
    system_prompt=SYSTEM_PROMPT.format(
        name="Alice", end_phrase=END_PHRASE, pov_response=pov(agent_name="Alice")
    ),
    # tools=[pov],
)

agent2 = Agent(
    name="Bob",
    system_prompt=SYSTEM_PROMPT.format(
        name="Bob", end_phrase=END_PHRASE, pov_response=pov(agent_name="Bob")
    ),
    # tools=[pov],
)

initial_message = agent1.initial_message(
    other=agent2,
    system_message=INITIALIZE_CONVERSATION_PROMPT.format(agent2.name),
    max_turns=2,
)
agent1.conversations[agent2.name]